In [1]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pyodbc
import pandas as pd

## connect to microsoft-sql-server through pyodbc

In [24]:

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DN-MSHERIF\SQLEXPRESS;'
                      'Database=Demo;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()



## Retrieve SQL Server metadata

In [25]:

sql_query_tables = "SELECT * FROM INFORMATION_SCHEMA.TABLES"  
cursor.execute(sql_query_tables)

metadata_results = cursor.fetchall()
table_name=''
tables_list =[]
for metadata_row in metadata_results:
    table_name = metadata_row.TABLE_NAME
    tables_list.append(table_name)
    

In [28]:
metadata = []
for i in range(len(tables_list)):
    sql_query_columns = "SELECT column_name, data_type ,TABLE_NAME from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME ='" + str(tables_list[i]) + "' "
    cursor.execute(sql_query_columns)
    metadata_results = cursor.fetchall()
    metadata.append(metadata_results)
    

## connect to snowflake account

In [5]:
engine = create_engine(URL(
    account='UCLSGAD-LY00738',
    user = 'sherif1922',
    password = 'Mohamed1922',
    database = 'MIGRATION',
    schema = 'public',
    warehouse = 'COMPUTE_WH',
    role='ACCOUNTADMIN',
))



In [6]:
snowflake_conn = engine.connect()
results = snowflake_conn.execute('select current_version()').fetchone()
print(results[0])

7.17.0


## create tables from the meta-data fetched from sql-server at snowflake

In [29]:
create_stmt_list = []
for index in range(len(metadata)):
    create_table_sql = f"CREATE OR REPLACE TABLE {metadata[index][0].TABLE_NAME} ("
    for row in metadata[index]:
            column_name, data_type ,table_name = row
            create_table_sql += f" {column_name} {data_type}, "
            #table_index+=1
    create_table_sql = create_table_sql.rstrip(", ") + ")"
    create_stmt_list.append(create_table_sql)
    
for stmt in create_stmt_list:
    snowflake_conn.execute(stmt)




## retrieve the data from sql server

In [30]:
db_data_list = []
for i in range(len(tables_list)):
    sql_query = 'SELECT * FROM "' + tables_list[i] + '" '
    cursor.execute(sql_query)
    data = cursor.fetchall()
    db_data_list.append(data)


## migrate data to snowflake

In [31]:
insert_stmt_list = []
for index in range(len(tables_list)):
    
    for row in db_data_list[index]:
        insert_table_sql = f"INSERT INTO  {metadata[index][0].TABLE_NAME} values ("
        for col in row:
  
            if isinstance(col, str) :
                insert_table_sql += f"'{col}',"
            else:
                insert_table_sql += f" {col},"
            
        insert_table_sql = insert_table_sql.rstrip(",") + ")"
        insert_stmt_list.append(insert_table_sql)
for stmt in insert_stmt_list:
    snowflake_conn.execute(stmt)
